In [ ]:
import json, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from datetime import timedelta, datetime, time

from research.utils.data_access_utils import RDSAccessUtils

rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))

In [ ]:
pen_id = 56

query = """
    select date, site_id, pen_id, biomass_data
    from day_summaries a
    where a.pen_id = %i
    and a.date >= '2020-04-01';
""" % (pen_id, )

df = rds_access_utils.extract_from_database(query)

df.index = pd.to_datetime(day_summaries['date'])
df = day_summaries.sort_index()

In [ ]:
import datetime

df.iloc[0]['date'] + datetime.timedelta(days=1)

In [ ]:
rawAvgWeight = []
rawNumFish = []

for index, row in df.iterrows():
    rawAvgWeight.append(row['biomass_data']['rawData']['avgWeight'])
    rawNumFish.append(row['biomass_data']['rawData']['numFish'])
    
df['rawAvgWeight'] = rawAvgWeight
df['rawNumFish'] = rawNumFish


In [ ]:
row['biomass_data']

In [ ]:
lastRawAvgWeight = []

for index, row in df.iterrows():
    lastDate = row['date'] - datetime.timedelta(days=1)
    lastRow = df[(df.date == lastDate)]
    if len(lastRow) > 0:
        lastRawAvgWeight.append(lastRow.ix[0, 'rawAvgWeight'])
    else:
        lastRawAvgWeight.append(None)
        
df['lastRawAvgWeight'] = lastRawAvgWeight

In [ ]:
plt.scatter(df['rawAvgWeight'], df['lastRawAvgWeight'])

In [ ]:
df['growthRate'] = (df['rawAvgWeight'] - df['lastRawAvgWeight']) / df['lastRawAvgWeight']

In [ ]:
plt.plot(df['growthRate'])

In [ ]:
lastGrowthRate = []

for index, row in df.iterrows():
    lastDate = row['date'] - datetime.timedelta(days=1)
    lastRow = df[(df.date == lastDate)]
    if len(lastRow) > 0:
        lastGrowthRate.append(lastRow.ix[0, 'growthRate'])
    else:
        lastGrowthRate.append(None)
        
df['lastGrowthRate'] = lastGrowthRate

In [ ]:
lastLastGrowthRate = []

for index, row in df.iterrows():
    lastDate = row['date'] - datetime.timedelta(days=1)
    lastRow = df[(df.date == lastDate)]
    if len(lastRow) > 0:
        lastLastGrowthRate.append(lastRow.ix[0, 'lastGrowthRate'])
    else:
        lastLastGrowthRate.append(None)
        
df['last2GrowthRate'] = lastLastGrowthRate

In [ ]:
lastLastGrowthRate = []

for index, row in df.iterrows():
    lastDate = row['date'] - datetime.timedelta(days=1)
    lastRow = df[(df.date == lastDate)]
    if len(lastRow) > 0:
        lastLastGrowthRate.append(lastRow.ix[0, 'last2GrowthRate'])
    else:
        lastLastGrowthRate.append(None)
        
df['last3GrowthRate'] = lastLastGrowthRate

In [ ]:
lastLastGrowthRate = []

for index, row in df.iterrows():
    lastDate = row['date'] - datetime.timedelta(days=1)
    lastRow = df[(df.date == lastDate)]
    if len(lastRow) > 0:
        lastLastGrowthRate.append(lastRow.ix[0, 'last3GrowthRate'])
    else:
        lastLastGrowthRate.append(None)
        
df['last4GrowthRate'] = lastLastGrowthRate

In [ ]:
lastLastGrowthRate = []

for index, row in df.iterrows():
    lastDate = row['date'] - datetime.timedelta(days=1)
    lastRow = df[(df.date == lastDate)]
    if len(lastRow) > 0:
        lastLastGrowthRate.append(lastRow.ix[0, 'last4GrowthRate'])
    else:
        lastLastGrowthRate.append(None)
        
df['last5GrowthRate'] = lastLastGrowthRate

In [ ]:
plt.scatter(df['lastGrowthRate'], df['growthRate'])

In [ ]:
import statsmodels.api as sm

df2 = df[(np.isfinite(df['last5GrowthRate'])) & (np.isfinite(df['last4GrowthRate'])) & (np.isfinite(df['last3GrowthRate'])) & (np.isfinite(df['last2GrowthRate'])) & (np.isfinite(df['lastGrowthRate'])) & (np.isfinite(df['growthRate']))]

X = df2[['lastGrowthRate', 'last2GrowthRate', 'last3GrowthRate', 'last4GrowthRate', 'last5GrowthRate']]
X = sm.add_constant(X)
y = df2['growthRate']
model = sm.OLS(y, X)
results = model.fit()

print(results.summary())

In [ ]:
(np.isfinite(df['lastGrowthRate'])) & (df['growthRate'] is not None)

In [ ]:
df['lastGrowthRate']